## Loading a custom mode


In [1]:
from mycelia.shared.modeling.custom_qwen3_next import CustomQwen3NextModel, get_moe_model_config
from mycelia.shared.config import MinerConfig
from mycelia.shared.expert_manager import ExpertManager

import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

import json
import torch
from __future__ import annotations
from transformers import AutoTokenizer, AutoModelForCausalLM
from itertools import chain
from collections import Counter
import matplotlib.pyplot as plt 

from mycelia.shared.helper import route_tokens_to_experts
from mycelia.shared.config import ValidatorConfig
from mycelia.shared.dataloader import get_dataloader
from mycelia.shared.expert_manager import is_expert_param
from mycelia.shared.modeling.mycelia import get_layer_expert_id

/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version 2.9.1 available.


In [2]:
# ---- Get config ----
rank = 0
config = ValidatorConfig()
config.task.data.batch_size = 1
config.task.data.sequence_length = 100

2025-11-20 10:35:19 [warning  ] Cannot find the wallet key by name coldkey name: template_coldkey_name, hotkey name: template_hotkey_name, please make sure it has been set correctly if you are reloading from a config.json or use --hotkey_name & --coldkey_name when you are creating a config file from a template. [mycelia.shared.config] error=Generic error: Failed to get hotkey: FileNotFound("Keyfile at: /home/isabella/.bittensor/wallets/template_coldkey_name/hotkeys/template_hotkey_name does not exist.")


In [3]:
config = MinerConfig()
em = ExpertManager(config)

2025-11-20 10:35:22 [warning  ] Cannot find the wallet key by name coldkey name: template_coldkey_name, hotkey name: template_hotkey_name, please make sure it has been set correctly if you are reloading from a config.json or use --hotkey_name & --coldkey_name when you are creating a config file from a template. [mycelia.shared.config] error=Generic error: Failed to get hotkey: FileNotFound("Keyfile at: /home/isabella/.bittensor/wallets/template_coldkey_name/hotkeys/template_hotkey_name does not exist.")
2025-11-20 10:35:22 [info     ] loading task folder          [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-11-20 10:35:22 [info     ] loading task folder          [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)


In [ ]:
topk = 10
group_ids = None
moe_config = get_moe_model_config(config, topk, group_ids, em.expert_group_assignment)

# get full model
model = CustomQwen3NextModel(moe_config)

In [ ]:
# get partial model
topk = 10
group_ids = [0]
moe_config = get_moe_model_config(config, topk, group_ids, em.expert_group_assignment)

# get partial model
moe_config.num_experts = em.num_experts
partial_model = CustomQwen3NextModel(moe_config)


## Model check

In [5]:
gate = model.layers[0].mlp.gate # .available_experts

In [8]:
_,  routing_weights, selected_experts = gate.forward(torch.rand(200, 2048))

expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes=25).permute(2, 1, 0)

expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero()
# routing_weights.sum(dim = 0).shape, expert_hit
expert_mask.sum(dim = 1)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 1,  ..., 1, 1, 0]])

## Test forward 

In [9]:
# ---- Get tokenizer ---- 
tokenizer = AutoTokenizer.from_pretrained(config.model.model_path)

# ---- Get dataloader ----
train_dataloader = get_dataloader(config, rank=rank, world_size=config.task.data.world_size, tokenizer=tokenizer)
iter_dataloader = iter(train_dataloader)

Too many dataloader workers: 4 (max is dataset.n_shards=1). Stopping 3 dataloader workers.


Too many dataloader workers: 4 (max is dataset.n_shards=1). Stopping 3 dataloader workers.


In [10]:
model.eval()
outputs = []
for i in range(20):
    tokens = next(iter_dataloader)
    for k, v in tokens.items():
        tokens[k] = v[0]
    del tokens['labels']
    with torch.no_grad():
        tokens.to(model.device)
        output = model(**tokens)
    outputs.append(output)

expert keys dict_keys(['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'])
expert_hit tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21],
        [22],
        [23],
        [24]])
expert keys dict_keys(['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'])
expert_hit tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21],
        [22],
        [23],
        [24]]

## loading a model 

In [ ]:
import asyncio
import copy
import gc
import os
import secrets
import time
from typing import Any, Dict, List, Tuple

import bittensor
import torch
import torch.nn as nn
from hivemind.averaging import DecentralizedAverager
from torchdata.stateful_dataloader import StatefulDataLoader
from transformers import PreTrainedTokenizerBase

from mycelia.miner.train_helper import get_status
from mycelia.shared.app_logging import configure_logging, structlog
from mycelia.shared.chain import ValidatorChainCommit, commit_status, setup_chain_worker
from mycelia.shared.checkpoint import (
    delete_old_checkpoints,
    get_resume_info,
    load_checkpoint,
    save_checkpoint,
    ModelMeta
)
from mycelia.shared.config import ValidatorConfig, parse_args
from mycelia.shared.cycle import gather_validation_job, should_start_validation
from mycelia.shared.dataloader import get_dataloader
from mycelia.shared.evaluate import evaluate_model
from mycelia.shared.expert_manager import (
    ExpertManager,
    get_weight_sum,
    populate_global_grads_from_local,
)
from mycelia.shared.helper import *
from mycelia.shared.metrics import MetricLogger
from mycelia.shared.model import load_model
from mycelia.shared.modeling.mycelia import get_base_tokenizer
from mycelia.validator.aggregator import MinerScoreAggregator
from mycelia.validator.evaluator import (
    MinerEvalJob,
    load_model_from_path,
    run_evaluation,
)
from mycelia.validator.inter_validator_connection import (
    build_averagers_from_buff,
    build_grad_buff_from_model,
    connect_with_peers,
    pack_grads,
    unpack_to_grads,
)
from mycelia.validator.rum import setup_training
config = ValidatorConfig.from_path('/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml')
rank = 0 

# === set up chain worker ===
wallet, subtensor = setup_chain_worker(config)

# === set logging ===
metric_logger = MetricLogger(config, rank)

# === mis ===
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(config)

eval_dataloader = get_dataloader(
    config,
    rank=0,
    world_size=10,
    tokenizer=tokenizer,
)

# === set up training ===
(
    base_model,
    global_model,
    outer_optimizer,
    outer_scaler,
    start_step,
    expert_manager,
    train_dataloader,
) = setup_training(config, rank, device, tokenizer, subtensor, wallet, current_model_meta = None)

2025-12-05 09:12:21 [info     ] found existing config path /home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml [mycelia.shared.config]


NameError: name 'setup_training' is not defined